<a href="https://colab.research.google.com/github/rajaonsonella/crosstalk-q2-2025/blob/main/notebooks/CROSSTALK_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

⚙️ Step 1: Set your notebook to GPU

The next two cells take ~2 min.... start running them now while we talk! 👇👇

In [51]:
# get workshop code
import os
import sys
IN_COLAB = os.getenv("COLAB_RELEASE_TAG")
if IN_COLAB:
    !git clone https://github.com/rajaonsonella/crosstalk-q2-2025
    sys.path.append('./crosstalk-q2-2025')
else:
    sys.path.append('..')
!pip install -r crosstalk-q2-2025/requirements.txt

fatal: destination path 'crosstalk-q2-2025' already exists and is not an empty directory.


In [52]:
# Download data from google drive
import gdown
import os

file_ids = {'test' : '19oR_A2UNUvy7pyL3J11oixkmF1m-bZBl',
            'train':'11S5p0QgP1X9rOFiIjNSLydLenJwm7hle'}

for name, file_id in file_ids.items():
    filename = f'crosstalk_{name}.parquet'
    if not os.path.exists(filename):
        gdown.download(id=file_id, output=filename, quiet=False)

# Load the train datasets

See the bonus content from last notebook to get a peek under the hood of the data loaders

Or check it out in the files you downloaded to colab on the left 👈

In [53]:
import pandas as pd
from dataset import basic_dataloader

In [54]:
X_train, y_train = basic_dataloader('crosstalk_train.parquet', x_col="AVALON", y_col = 'DELLabel', n_to_load=1000) # fingerprints available: 'ATOMPAIR', 'MACCS', 'ECFP6', 'ECFP4', 'FCFP4', 'FCFP6', 'TOPTOR', 'RDK', 'AVALON'

In [55]:
print(X_train)

[[ 0  0  0 ...  0  1  3]
 [ 0  0  0 ...  0  5 12]
 [ 1  0  0 ...  0  1  7]
 ...
 [ 1  1  0 ...  0  0  6]
 [ 0  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  1  4]]


In [56]:
X_train.shape

(1000, 2048)

In [57]:
print(y_train)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 1 0 

In [58]:
y_train.shape

(1000,)

# Let's train catboost classifier and see how well it fits the training data

🐞 do you see a CUDA error? raise your hand now and brag about it

In [59]:
%%time
import catboost as cb
from eval import BinaryEvaluator
params = {
                'random_strength': 2, # only non-default hyperparam, default is 1
                'random_seed': 1234,
                'verbose': 0,
                'loss_function': 'Logloss',
                'task_type': 'GPU',
                'devices': '0'
            }
model = cb.CatBoostClassifier(**params)
eval = BinaryEvaluator(X_train, y_train)
model.fit(X_train, y_train)
yp = model.predict_proba(X_train)[:, 1] # or validation
metric_dict = eval.compute_metrics(yt=y_train, yp=yp) # or validation

KeyboardInterrupt: 

In [60]:
for metric_name, metric_value in metric_dict.items():
    print(f'{metric_name:20s}: {metric_value:.2f}')

accuracy            : 1.00
balanced_accuracy   : 1.00
roc_auc             : 1.00
precision           : 1.00
recall              : 1.00
mean_reciprocal_rank: 0.08
positives           : 59.00
predicted_positives : 59.00
hits_at_5           : 0.08
precision_at_5      : 1.00
hits_at_10          : 0.17
precision_at_10     : 1.00
hits_at_30          : 0.51
precision_at_30     : 1.00
hits_at_59          : 1.00
precision_at_59     : 1.00


# How well does it generalize though? Let's try 5-fold cross-validation

⚠️ these next cells are slow to run! Start them now and come back in 5 minutes

In [41]:
%%time
model = cb.CatBoostClassifier(**params)
metric_dict_cv = eval.CV_model(model)

CPU times: user 3min 6s, sys: 21 s, total: 3min 27s
Wall time: 3min 1s


In [42]:
for metric_name, metric_value in metric_dict_cv['mean'].items():
    print(f'{metric_name:20s}: {metric_value:.2f}')

accuracy            : 0.95
balanced_accuracy   : 0.60
roc_auc             : 0.79
precision           : 0.82
recall              : 0.20
mrr                 : 0.16
precision_at_k_5    : 0.56
hits_at_k_5         : 0.24
precision_at_k_10   : 0.46
hits_at_k_10        : 0.39
precision_at_k_30   : 0.23
hits_at_k_30        : 0.59


# Let's compare it against simpler sklearn baselines

In [43]:
%%time
from eval import get_baseline_models

eval = BinaryEvaluator(X_train, y_train)
baselines = get_baseline_models()
baselines_res = {}

for m in baselines:
    baselines_res[m] = eval.CV_model(baselines[m])

CPU times: user 42.8 s, sys: 60.9 ms, total: 42.9 s
Wall time: 26 s


In [44]:
# display all the models results
baselines_res.update({'catboost': metric_dict_cv})
pd.DataFrame({model: metrics['mean'] for model, metrics in baselines_res.items()}).T.round(2)

,accuracy,balanced_accuracy,roc_auc,precision,recall,mrr,precision_at_k_5,hits_at_k_5,precision_at_k_10,hits_at_k_10,precision_at_k_30,hits_at_k_30
stratified_dummy,0.88,0.50,0.50,0.05,0.07,0.14,0.04,0.07,0.04,0.27,0.03,1.00
most_frequent_dummy,0.94,0.50,0.50,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.01,1.00
uniform_dummy,0.06,0.50,0.50,0.06,1.00,1.00,0.00,1.00,0.00,1.00,0.01,1.00
logistic_regression,0.92,0.62,0.74,0.32,0.29,0.09,0.36,0.15,0.30,0.25,0.19,0.47
decision_tree,0.92,0.51,0.51,0.13,0.05,0.16,0.24,0.15,0.16,0.75,0.06,0.90
random_forest,0.94,0.51,0.79,0.20,0.02,0.11,0.44,0.19,0.34,0.29,0.19,0.49
linear_svc,0.94,0.50,0.73,0.00,0.00,0.08,0.24,0.10,0.26,0.22,0.17,0.44
catboost,0.95,0.60,0.79,0.82,0.20,0.16,0.56,0.24,0.46,0.39,0.23,0.59


# Submit predictions

Update the next cell with your team name

In [45]:
team_name = 'demo'

In [46]:
%%time
X_test = basic_dataloader('crosstalk_test.parquet', x_col="AVALON", y_col = None, max_to_load = 400000, chunk_size = 20000)

TypeError: basic_dataloader() got an unexpected keyword argument 'max_to_load'

In [47]:
X_test.shape

(339204, 2048)

In [48]:
yp = model.predict_proba(X_test)[:,1]
pd.write_csv(f'{team_name}.csv', yp)

CatBoostError: There is no trained model to use predict_proba(). Use fit() to train model. Then use this method.

Upload this baseline to kaggle and check out the leaderboard!